# Simple RNN Tensorflow

In [33]:
# this cell is for imports
from __future__ import absolute_import, division, print_function

import tensorflow as tf
tf.enable_eager_execution()

import numpy as np
import pandas as pd
import collections
import os
import time
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/shunlin/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [34]:
# getting jokes from kaggle dataset
kaggle_jokes_raw = pd.read_csv('input/kaggle/shortjokes.csv')

In [35]:
# see the head of the jokes
kaggle_jokes_raw.head()

,ID,Joke
0,1,"[me narrating a documentary about narrators] ""..."
1,2,Telling my daughter garlic is good for you. Go...
2,3,I've been going through a really rough period ...
3,4,"If I could have dinner with anyone, dead or al..."
4,5,Two guys walk into a bar. The third guy ducks.


In [42]:
# the rest of this notebook follows code from https://www.tensorflow.org/tutorials/sequences/text_generation
# build a set of vocab (characters)
joke_lst = kaggle_jokes_raw["Joke"].tolist()
text = joke_lst[0]
# use the first 1000 jokes to play with model
for joke in joke_lst[1:20000]:
    text += '\n\n'
    text += joke
vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))

97 unique characters


In [43]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [44]:
print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

{
  '\x08':   0,
  '\n':   1,
  ' ' :   2,
  '!' :   3,
  '"' :   4,
  '#' :   5,
  '$' :   6,
  '%' :   7,
  '&' :   8,
  "'" :   9,
  '(' :  10,
  ')' :  11,
  '*' :  12,
  '+' :  13,
  ',' :  14,
  '-' :  15,
  '.' :  16,
  '/' :  17,
  '0' :  18,
  '1' :  19,
  ...
}


In [45]:
# Show how the first 13 characters from the text are mapped to integers
print ('{} ---- characters mapped to int ---- > {}'.format(repr(text[:13]), text_as_int[:13]))

'[me narrating' ---- characters mapped to int ---- > [61 79 71  2 80 67 84 84 67 86 75 80 73]


In [46]:
# The maximum length sentence we want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//seq_length

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
    print(idx2char[i.numpy()])

[
m
e
 
n


In [47]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
    print(repr(''.join(idx2char[item.numpy()])))

'[me narrating a documentary about narrators] "I can\'t hear what they\'re saying cuz I\'m talking"\n\nTell'
'ing my daughter garlic is good for you. Good immune system and keeps pests away.Ticks, mosquitos, vam'
"pires... men.\n\nI've been going through a really rough period at work this week It's my own fault for "
'swapping my tampax for sand paper.\n\nIf I could have dinner with anyone, dead or alive... ...I would c'
"hoose alive. -B.J. Novak-\n\nTwo guys walk into a bar. The third guy ducks.\n\nWhy can't Barbie get pregn"


In [48]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [49]:
for input_example, target_example in  dataset.take(1):
    print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
    print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  '[me narrating a documentary about narrators] "I can\'t hear what they\'re saying cuz I\'m talking"\n\nTel'
Target data: 'me narrating a documentary about narrators] "I can\'t hear what they\'re saying cuz I\'m talking"\n\nTell'


In [50]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
  input: 61 ('[')
  expected output: 79 ('m')
Step    1
  input: 79 ('m')
  expected output: 71 ('e')
Step    2
  input: 71 ('e')
  expected output: 2 (' ')
Step    3
  input: 2 (' ')
  expected output: 80 ('n')
Step    4
  input: 80 ('n')
  expected output: 67 ('a')


In [51]:
# Batch size 
BATCH_SIZE = 64
steps_per_epoch = examples_per_epoch//BATCH_SIZE

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences, 
# so it doesn't attempt to shuffle the entire sequence in memory. Instead, 
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<DatasetV1Adapter shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

In [52]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension 
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [53]:
if tf.test.is_gpu_available():
    rnn = tf.keras.layers.CuDNNGRU
else:
    import functools
    rnn = functools.partial(
        tf.keras.layers.GRU, recurrent_activation='sigmoid')

In [54]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim, 
                              batch_input_shape=[batch_size, None]),
        rnn(rnn_units,
            return_sequences=True, 
            recurrent_initializer='glorot_uniform',
            stateful=True),
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [55]:
model = build_model(
  vocab_size = len(vocab), 
  embedding_dim=embedding_dim, 
  rnn_units=rnn_units, 
  batch_size=BATCH_SIZE)

In [56]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 97) # (batch_size, sequence_length, vocab_size)


In [57]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (64, None, 256)           24832     
_________________________________________________________________
gru_2 (GRU)                  (64, None, 1024)          3935232   
_________________________________________________________________
dense_2 (Dense)              (64, None, 97)            99425     
Total params: 4,059,489
Trainable params: 4,059,489
Non-trainable params: 0
_________________________________________________________________


In [58]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [59]:
sampled_indices

array([21, 48,  5, 16, 59, 25, 27, 71, 77, 65,  1, 88, 21, 34, 34,  6, 49,
       13,  1, 67, 69, 74, 19, 60,  9, 44, 94, 88, 24, 92, 51,  9, 90, 33,
       52, 67, 88, 31, 70, 25, 88, 39, 76, 28, 82, 74, 66, 45, 90, 61, 53,
       59, 22, 78, 34, 34, 12, 93, 32, 10, 67, 81, 50, 93, 88, 62, 76,  5,
       46, 21, 70, 41, 36,  0, 37, 42, 29, 16, 66, 31,  5, 15, 52, 61, 93,
       53, 65, 42, 34,  2, 85, 41, 72, 28, 74,  5, 32,  6,  8, 28])

In [60]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 'o many exotic sounding flavours these days. I just can\'t resist-" Doctor: "YOU NEED TO STOP DRINKING'

Next Char Predictions: 
 "3N#.Y79ek_\nv3@@$O+\nach1Z'J|v6zQ'x?Rav=d7vEj:ph`Kx[SY4l@@*{>(aoP{v\\j#L3dGB\x08CH;.`=#-R[{S_H@ sGf:h#>$&:"


In [61]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)") 
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 97)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.576399


In [62]:
model.compile(
    optimizer = tf.train.AdamOptimizer(),
    loss = loss)

In [63]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [64]:
EPOCHS=10

In [65]:
history = model.fit(dataset.repeat(), epochs=EPOCHS, steps_per_epoch=steps_per_epoch, callbacks=[checkpoint_callback])


Epoch 1/10
298/298 [==============================] - 2077s 7s/step - loss: 2.5149
Epoch 2/10
298/298 [==============================] - 1996s 7s/step - loss: 1.8531
Epoch 3/10
298/298 [==============================] - 1802s 6s/step - loss: 1.6350
Epoch 4/10
298/298 [==============================] - 1828s 6s/step - loss: 1.5182
Epoch 5/10
298/298 [==============================] - 1737s 6s/step - loss: 1.4417
Epoch 6/10
298/298 [==============================] - 1739s 6s/step - loss: 1.3854
Epoch 7/10
298/298 [==============================] - 1736s 6s/step - loss: 1.3355
Epoch 8/10
298/298 [==============================] - 1740s 6s/step - loss: 1.2907
Epoch 9/10
298/298 [==============================] - 1744s 6s/step - loss: 1.2478
Epoch 10/10
298/298 [==============================] - 1735s 6s/step - loss: 1.2064


In [66]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_10'

In [67]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [68]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (1, None, 256)            24832     
_________________________________________________________________
gru_3 (GRU)                  (1, None, 1024)           3935232   
_________________________________________________________________
dense_3 (Dense)              (1, None, 97)             99425     
Total params: 4,059,489
Trainable params: 4,059,489
Non-trainable params: 0
_________________________________________________________________


In [84]:
def generate_text(model, start_string):
    # Evaluation step (generating text using the learned model)

    # Number of characters to generate
    num_generate = 500

    # Converting our start string to numbers (vectorizing) 
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # Empty string to store our results
    text_generated = []

    # Low temperatures results in more predictable text.
    # Higher temperatures results in more surprising text.
    # Experiment to find the best setting.
    temperature = 0.4

    # Here batch size == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        # remove the batch dimension
        predictions = tf.squeeze(predictions, 0)

        # using a multinomial distribution to predict the word returned by the model
        predictions = predictions / temperature
        predicted_id = tf.multinomial(predictions, num_samples=1)[-1,0].numpy()

        # We pass the predicted word as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))

In [85]:
print(generate_text(model, start_string=u"Why did the chicken "))

Why did the chicken cross the road? Because he's not an order with a big dick.

What do you call a black man flying a bank with a baby conversation with a short one.

The defendant says "we don't want to walk a haircut the wall." - We are correcting them but he didn't have a headache. Master addicted to the vet banned from the back of the movie about a gun and the other was a head of the restaurant on the morning.

What do you call a black man who comes out and start saying "we don't have to be the first time on my
